In [8]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as FS
from pyspark.sql import types as T

In [3]:
spark = SparkSession.builder.master("local[2]").appName("JSON_PARSER").getOrCreate()

In [24]:
data=[('Khan','{"street": "Sadar Bazar", "city": "Meerut"}'),('Rahul','{"street": "Hall Town", "city": "Kanpur"}')]
df=spark.createDataFrame(data,schema="name string,address string")
df.show()

+-----+--------------------+
| name|             address|
+-----+--------------------+
| Khan|{"street": "Sadar...|
|Rahul|{"street": "Hall ...|
+-----+--------------------+



**Convert the above dataframe format into below one**
```
+-----+-----------+------+
| name|     street|  city|
+-----+-----------+------+
| Khan|Sadar Bazar|Meerut|
|Rahul|  Hall Town|Kanpur|
+-----+-----------+------+
```

df.printSchema()

In [26]:
#schema to represent out json data
schema = T.StructType(
  [
    T.StructField('street', T.StringType(), True),
    T.StructField('city', T.StringType(), True)
  ]
)

**Conver address String type to json type**

In [27]:
df2 = df.withColumn("address",FS.from_json("address",schema))

In [28]:
df2.printSchema()

root
 |-- name: string (nullable = true)
 |-- address: struct (nullable = true)
 |    |-- street: string (nullable = true)
 |    |-- city: string (nullable = true)



In [29]:
df2.show()

+-----+--------------------+
| name|             address|
+-----+--------------------+
| Khan|{Sadar Bazar, Mee...|
|Rahul| {Hall Town, Kanpur}|
+-----+--------------------+



**Now get the json field to show the result**

In [31]:
df2.select(df2.name,df2.address.street.alias('street'),df2.address.city.alias('city')).show()

+-----+-----------+------+
| name|     street|  city|
+-----+-----------+------+
| Khan|Sadar Bazar|Meerut|
|Rahul|  Hall Town|Kanpur|
+-----+-----------+------+

